### Import packages

In [2]:
!pip install rasterio

     |████████████████████████████████| 15.0MB 4.4MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
!pip install tqdm==4.33.0

     |████████████████████████████████| 51kB 3.1MB/s eta 0:00:011
  Found existing installation: tqdm 4.31.1
    Uninstalling tqdm-4.31.1:
      Successfully uninstalled tqdm-4.31.1
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
!pip show tqdm

Name: tqdm
Version: 4.33.0
Summary: Fast, Extensible Progress Meter
Home-page: https://github.com/tqdm/tqdm
Author: None
Author-email: None
License: MPLv2.0, MIT Licences
Location: /data/anaconda/envs/py35/lib/python3.5/site-packages
Requires: 
Required-by: twine


In [6]:
!pip install geopandas

     |████████████████████████████████| 901kB 4.5MB/s eta 0:00:01
     |████████████████████████████████| 17.9MB 21.9MB/s eta 0:00:01
     |████████████████████████████████| 1.5MB 29.9MB/s eta 0:00:01
     |████████████████████████████████| 9.8MB 29.0MB/s eta 0:00:01
  Created wheel for munch: filename=munch-2.3.2-py2.py3-none-any.whl size=5793 sha256=06d60ffe5065038a7f371b526c167cd6e19e74cfbd33dbf4d4bb3435a5c1b0e6
  Stored in directory: /home/firou1925/.cache/pip/wheels/db/bf/bc/06a3e1bfe0ab27d2e720ceb3cff3159398d92644c0cec2c125
Successfully built munch
You should consider upgrading via the 'pip install --upgrade pip' command.


In [7]:
import os
import requests, zipfile, io
import numpy as np # linear algebra
import pandas as pd

import rasterio
from rasterio.plot import show
from rasterio.plot import show_hist
import matplotlib.pyplot as plt        
%matplotlib inline
from matplotlib.pyplot import imshow
from scipy import stats
from rasterio.merge import merge
import geopandas as gpd
from rasterio.mask import mask
from rasterio.plot import adjust_band
from rasterio.plot import reshape_as_raster, reshape_as_image
from shapely.geometry import mapping
import sys
from tqdm import tqdm
tqdm.pandas()
import gc
import psutil

from queue import PriorityQueue

## Import shapefiles

In [19]:
os.listdir('data/test')

['test.dbf',
 '.ipynb_checkpoints',
 'test.shp',
 'test.cpg',
 'test.prj',
 'test.shx',
 'test.qpj']

In [22]:
train = gpd.read_file('data/train/train.shp')
train.dropna(subset=['geometry'], inplace = True)
train.reset_index(drop=True, inplace= True)
test = gpd.read_file('data/test/test.shp')
sample_sub = pd.read_csv('data/sample_submission.csv')

train_fe = train.copy()
test_fe = test.copy()

In [21]:
train_fe.head()

,Field_Id,Area,Subregion,Crop_Id_Ne,geometry
0,1,0.915905,3,8,POLYGON ((2467881.175041331 -3352032.059296422...
1,2,2.064410,3,6,"POLYGON ((2468428.90623293 -3351993.17305059, ..."
2,3,1.080300,3,8,POLYGON ((2468333.745293362 -3352058.485056543...
3,4,1.316190,3,8,POLYGON ((2468607.857733211 -3352012.337531495...
4,7,5.529220,3,8,"POLYGON ((2468646.08520868 -3351706.201920062,..."


### Folder names

In [8]:
date1 = '2017-01-01'
date2 = "2017-01-31"
date3 = "2017-02-10"
date4 = "2017-03-12"
date5 = "2017-03-22"
date6 = "2017-05-31"
date7 = "2017-06-20"
date8 = "2017-07-10"
date9 = "2017-07-15"
date10 = "2017-08-04"
date11 = "2017-08-19"

In [12]:
date1_pf1 = 'S2A_MSIL1C_20170101T082332_N0204_R121_T34JEP_20170101T084543.SAFE'
date1_pf2 = 'S2A_MSIL1C_20170101T082332_N0204_R121_T34JFP_20170101T084543.SAFE'

date2_pf1 = 'S2A_MSIL1C_20170131T082151_N0204_R121_T34JEP_20170131T084118.SAFE'
date2_pf2 = 'S2A_MSIL1C_20170131T082151_N0204_R121_T34JFP_20170131T084118.SAFE'

date3_pf1 = 'S2A_MSIL1C_20170210T082051_N0204_R121_T34JEP_20170210T083752.SAFE'
date3_pf2 = 'S2A_MSIL1C_20170210T082051_N0204_R121_T34JFP_20170210T083752.SAFE'

date4_pf1 = 'S2A_MSIL1C_20170312T082001_N0204_R121_T34JEP_20170312T084235.SAFE '
date4_pf2 = 'S2A_MSIL1C_20170312T082001_N0204_R121_T34JFP_20170312T084235.SAFE'

date5_pf1 = 'S2A_MSIL1C_20170322T081611_N0204_R121_T34JEP_20170322T084728.SAFE'
date5_pf2 = 'S2A_MSIL1C_20170322T081611_N0204_R121_T34JFP_20170322T084728.SAFE'

date6_pf1 = 'S2A_MSIL1C_20170531T082011_N0205_R121_T34JEP_20170531T084246.SAFE'
date6_pf2 = 'S2A_MSIL1C_20170531T082011_N0205_R121_T34JFP_20170531T084246.SAFE'

date7_pf1 = 'S2A_MSIL1C_20170620T082011_N0205_R121_T34JEP_20170620T084200.SAFE'
date7_pf2 = 'S2A_MSIL1C_20170620T082011_N0205_R121_T34JFP_20170620T084200.SAFE'

date8_pf1 = 'S2A_MSIL1C_20170710T082011_N0205_R121_T34JEP_20170710T084244.SAFE'
date8_pf2 = 'S2A_MSIL1C_20170710T082011_N0205_R121_T34JFP_20170710T084244.SAFE'

date9_pf1 = 'S2B_MSIL1C_20170715T081609_N0205_R121_T34JEP_20170715T084650.SAFE'
date9_pf2 = 'S2B_MSIL1C_20170715T081609_N0205_R121_T34JFP_20170715T084650.SAFE'

date10_pf1 = 'S2B_MSIL1C_20170804T081559_N0205_R121_T34JEP_20170804T084631.SAFE'
date10_pf2 = 'S2B_MSIL1C_20170804T081559_N0205_R121_T34JFP_20170804T084631.SAFE'

date11_pf1 = 'S2A_MSIL1C_20170819T082011_N0205_R121_T34JEP_20170819T084427.SAFE'
date11_pf2 = 'S2A_MSIL1C_20170819T082011_N0205_R121_T34JFP_20170819T084427.SAFE'

In [17]:
for dirname, _, filenames in os.walk('data/'+date1_pf1):
    for filename in filenames:
        print(os.path.join(dirname, filename))

data/S2A_MSIL1C_20170101T082332_N0204_R121_T34JEP_20170101T084543.SAFE/INSPIRE.xml
data/S2A_MSIL1C_20170101T082332_N0204_R121_T34JEP_20170101T084543.SAFE/manifest.safe
data/S2A_MSIL1C_20170101T082332_N0204_R121_T34JEP_20170101T084543.SAFE/MTD_MSIL1C.xml
data/S2A_MSIL1C_20170101T082332_N0204_R121_T34JEP_20170101T084543.SAFE/rep_info/S2_User_Product_Level-1C_Metadata.xsd
data/S2A_MSIL1C_20170101T082332_N0204_R121_T34JEP_20170101T084543.SAFE/DATASTRIP/DS_MTI__20170103T030708_S20170101T084543/MTD_DS.xml
data/S2A_MSIL1C_20170101T082332_N0204_R121_T34JEP_20170101T084543.SAFE/HTML/star_bg.jpg
data/S2A_MSIL1C_20170101T082332_N0204_R121_T34JEP_20170101T084543.SAFE/HTML/UserProduct_index.xsl
data/S2A_MSIL1C_20170101T082332_N0204_R121_T34JEP_20170101T084543.SAFE/HTML/UserProduct_index.html
data/S2A_MSIL1C_20170101T082332_N0204_R121_T34JEP_20170101T084543.SAFE/HTML/banner_3.png
data/S2A_MSIL1C_20170101T082332_N0204_R121_T34JEP_20170101T084543.SAFE/HTML/banner_2.png
data/S2A_MSIL1C_20170101T082332_

## Create mosaic

In [24]:
def create_mosaic(fp_bands, sp_bands, date, directory = ''):
    """
    This function creates a mosaic from two raster images and saves it to the directory with the name Mosaic_"+ date+ ".tiff.
    The mosaic will have a number of bands equal to the ones specified in the fp_bands.
    fp_bands: list:
    list of first raster bands that we use to construct the first raster
    sp_bands: list:
    list of the second raster bands that we use to construct the second raster
    date: string:
    the date of the raster, we use it to name the final registered the mosaic
    """
    
    if(len(fp_bands) == 0 or len(sp_bands)== 0):
        print('There are no bands')
        return 0
    if(len(fp_bands) != len(sp_bands)):
        print('The length of the two bands lists are not equal')
        return 0
    
    ## set the first part of the mosaic image
    nb_bands = len(fp_bands)
    with rasterio.open(fp_bands[0]) as src0:
        fb_meta = src0.meta

    fb_meta['driver'] = 'Gtiff'
    fb_meta['count'] = nb_bands
    
    #export true color image
    with rasterio.open('first_part.tiff', 'w', **fb_meta) as dst:
        for id, layer in enumerate(fp_bands, start=1):
            with rasterio.open(layer) as src1:
                dst.write(src1.read(1), id)

    print('first part completed')
    
    ## set the second part of the mosaic image

    with rasterio.open(sp_bands[0]) as src0:
        sb_meta = src0.meta

    sb_meta['driver'] = 'Gtiff'
    sb_meta['count'] = nb_bands
    
    #export true color image
    with rasterio.open('second_part.tiff', 'w', **sb_meta) as dst:
        for id, layer in enumerate(sp_bands, start=1):
            with rasterio.open(layer) as src1:
                dst.write(src1.read(1), id)
    print('second part completed')
    
    ## Create the mosaic
    first_part = rasterio.open(r"first_part.tiff", count=nb_bands)
    second_part = rasterio.open(r"second_part.tiff", count=nb_bands)
    os.remove(r"./first_part.tiff")
    os.remove(r"./second_part.tiff")

    src_files_to_mosaic = [first_part, second_part]  
    mosaic, out_trans = merge(src_files_to_mosaic)
    
    out_meta = sb_meta.copy()
    out_meta.update({"height": mosaic.shape[1],
                    "width": mosaic.shape[2],
                    "transform": out_trans})

    out_fp = os.path.join('mosaics', r"Mosaic_"+ date+ ".tiff")
    with rasterio.open(out_fp, "w", **out_meta) as dest:
        dest.write(mosaic)
        
        
    first_part.close()
    second_part.close()
    gc.collect()
    print('mosaic is created and saved')

In [25]:
### 2017-01-01 mosiac

parent_path = 'data'

raster_01_01_2017_b2_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170101T082332_N0204_R121_T34JEP_20170101T084543.SAFE/GRANULE/L1C_T34JEP_A007983_20170101T084543/IMG_DATA/T34JEP_20170101T082332_B02.jp2')
raster_01_01_2017_b3_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170101T082332_N0204_R121_T34JEP_20170101T084543.SAFE/GRANULE/L1C_T34JEP_A007983_20170101T084543/IMG_DATA/T34JEP_20170101T082332_B03.jp2')
raster_01_01_2017_b4_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170101T082332_N0204_R121_T34JEP_20170101T084543.SAFE/GRANULE/L1C_T34JEP_A007983_20170101T084543/IMG_DATA/T34JEP_20170101T082332_B04.jp2')
raster_01_01_2017_b8_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170101T082332_N0204_R121_T34JEP_20170101T084543.SAFE/GRANULE/L1C_T34JEP_A007983_20170101T084543/IMG_DATA/T34JEP_20170101T082332_B08.jp2')
fp_bands = [raster_01_01_2017_b2_fp, raster_01_01_2017_b3_fp, raster_01_01_2017_b4_fp, raster_01_01_2017_b8_fp]


raster_01_01_2017_jfp_b2_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170101T082332_N0204_R121_T34JFP_20170101T084543.SAFE/GRANULE/L1C_T34JFP_A007983_20170101T084543/IMG_DATA/T34JFP_20170101T082332_B02.jp2')
raster_01_01_2017_jfp_b3_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170101T082332_N0204_R121_T34JFP_20170101T084543.SAFE/GRANULE/L1C_T34JFP_A007983_20170101T084543/IMG_DATA/T34JFP_20170101T082332_B03.jp2')
raster_01_01_2017_jfp_b4_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170101T082332_N0204_R121_T34JFP_20170101T084543.SAFE/GRANULE/L1C_T34JFP_A007983_20170101T084543/IMG_DATA/T34JFP_20170101T082332_B04.jp2')
raster_01_01_2017_jfp_b8_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170101T082332_N0204_R121_T34JFP_20170101T084543.SAFE/GRANULE/L1C_T34JFP_A007983_20170101T084543/IMG_DATA/T34JFP_20170101T082332_B08.jp2')
sp_bands = [raster_01_01_2017_jfp_b2_fp, raster_01_01_2017_jfp_b3_fp, raster_01_01_2017_jfp_b4_fp, raster_01_01_2017_jfp_b8_fp]

date = '2017-01-01'
create_mosaic(fp_bands, sp_bands, date)

first part completed
second part completed
mosaic is created and saved


In [26]:
### 2017-01-31 mosiac

raster_b2_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170131T082151_N0204_R121_T34JEP_20170131T084118.SAFE/GRANULE/L1C_T34JEP_A008412_20170131T084118/IMG_DATA/T34JEP_20170131T082151_B02.jp2')
raster_b3_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170131T082151_N0204_R121_T34JEP_20170131T084118.SAFE/GRANULE/L1C_T34JEP_A008412_20170131T084118/IMG_DATA/T34JEP_20170131T082151_B03.jp2')
raster_b4_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170131T082151_N0204_R121_T34JEP_20170131T084118.SAFE/GRANULE/L1C_T34JEP_A008412_20170131T084118/IMG_DATA/T34JEP_20170131T082151_B04.jp2')
raster_b8_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170131T082151_N0204_R121_T34JEP_20170131T084118.SAFE/GRANULE/L1C_T34JEP_A008412_20170131T084118/IMG_DATA/T34JEP_20170131T082151_B08.jp2')
fp_bands = [raster_b2_fp, raster_b3_fp, raster_b4_fp, raster_b8_fp]


raster_jfp_b2_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170131T082151_N0204_R121_T34JFP_20170131T084118.SAFE/GRANULE/L1C_T34JFP_A008412_20170131T084118/IMG_DATA/T34JFP_20170131T082151_B02.jp2')
raster_jfp_b3_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170131T082151_N0204_R121_T34JFP_20170131T084118.SAFE/GRANULE/L1C_T34JFP_A008412_20170131T084118/IMG_DATA/T34JFP_20170131T082151_B03.jp2')
raster_jfp_b4_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170131T082151_N0204_R121_T34JFP_20170131T084118.SAFE/GRANULE/L1C_T34JFP_A008412_20170131T084118/IMG_DATA/T34JFP_20170131T082151_B04.jp2')
raster_jfp_b8_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170131T082151_N0204_R121_T34JFP_20170131T084118.SAFE/GRANULE/L1C_T34JFP_A008412_20170131T084118/IMG_DATA/T34JFP_20170131T082151_B08.jp2')
sp_bands = [raster_jfp_b2_fp, raster_jfp_b3_fp, raster_jfp_b4_fp, raster_jfp_b8_fp]

date = '2017-01-31'
create_mosaic(fp_bands, sp_bands, date)

first part completed
second part completed
mosaic is created and saved


In [28]:
### 2017-02-10 mosiac

raster_b2_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170210T082051_N0204_R121_T34JEP_20170210T083752.SAFE/GRANULE/L1C_T34JEP_A008555_20170210T083752/IMG_DATA/T34JEP_20170210T082051_B02.jp2')
raster_b3_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170210T082051_N0204_R121_T34JEP_20170210T083752.SAFE/GRANULE/L1C_T34JEP_A008555_20170210T083752/IMG_DATA/T34JEP_20170210T082051_B03.jp2')
raster_b4_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170210T082051_N0204_R121_T34JEP_20170210T083752.SAFE/GRANULE/L1C_T34JEP_A008555_20170210T083752/IMG_DATA/T34JEP_20170210T082051_B04.jp2')
raster_b8_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170210T082051_N0204_R121_T34JEP_20170210T083752.SAFE/GRANULE/L1C_T34JEP_A008555_20170210T083752/IMG_DATA/T34JEP_20170210T082051_B08.jp2')
fp_bands = [raster_b2_fp, raster_b3_fp, raster_b4_fp, raster_b8_fp]


raster_jfp_b2_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170210T082051_N0204_R121_T34JFP_20170210T083752.SAFE/GRANULE/L1C_T34JFP_A008555_20170210T083752/IMG_DATA/T34JFP_20170210T082051_B02.jp2')
raster_jfp_b3_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170210T082051_N0204_R121_T34JFP_20170210T083752.SAFE/GRANULE/L1C_T34JFP_A008555_20170210T083752/IMG_DATA/T34JFP_20170210T082051_B03.jp2')
raster_jfp_b4_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170210T082051_N0204_R121_T34JFP_20170210T083752.SAFE/GRANULE/L1C_T34JFP_A008555_20170210T083752/IMG_DATA/T34JFP_20170210T082051_B04.jp2')
raster_jfp_b8_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170210T082051_N0204_R121_T34JFP_20170210T083752.SAFE/GRANULE/L1C_T34JFP_A008555_20170210T083752/IMG_DATA/T34JFP_20170210T082051_B08.jp2')
sp_bands = [raster_jfp_b2_fp, raster_jfp_b3_fp, raster_jfp_b4_fp, raster_jfp_b8_fp]

date = '2017-02-10'
create_mosaic(fp_bands, sp_bands, date)

first part completed
second part completed
mosaic is created and saved


In [29]:
### 2017-03-12 mosiac

raster_b2_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170312T082001_N0204_R121_T34JEP_20170312T084235.SAFE/GRANULE/L1C_T34JEP_A008984_20170312T084235/IMG_DATA/T34JEP_20170312T082001_B02.jp2')
raster_b3_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170312T082001_N0204_R121_T34JEP_20170312T084235.SAFE/GRANULE/L1C_T34JEP_A008984_20170312T084235/IMG_DATA/T34JEP_20170312T082001_B03.jp2')
raster_b4_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170312T082001_N0204_R121_T34JEP_20170312T084235.SAFE/GRANULE/L1C_T34JEP_A008984_20170312T084235/IMG_DATA/T34JEP_20170312T082001_B04.jp2')
raster_b8_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170312T082001_N0204_R121_T34JEP_20170312T084235.SAFE/GRANULE/L1C_T34JEP_A008984_20170312T084235/IMG_DATA/T34JEP_20170312T082001_B08.jp2')
fp_bands = [raster_b2_fp, raster_b3_fp, raster_b4_fp, raster_b8_fp]


raster_jfp_b2_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170312T082001_N0204_R121_T34JFP_20170312T084235.SAFE/GRANULE/L1C_T34JFP_A008984_20170312T084235/IMG_DATA/T34JFP_20170312T082001_B02.jp2')
raster_jfp_b3_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170312T082001_N0204_R121_T34JFP_20170312T084235.SAFE/GRANULE/L1C_T34JFP_A008984_20170312T084235/IMG_DATA/T34JFP_20170312T082001_B03.jp2')
raster_jfp_b4_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170312T082001_N0204_R121_T34JFP_20170312T084235.SAFE/GRANULE/L1C_T34JFP_A008984_20170312T084235/IMG_DATA/T34JFP_20170312T082001_B04.jp2')
raster_jfp_b8_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170312T082001_N0204_R121_T34JFP_20170312T084235.SAFE/GRANULE/L1C_T34JFP_A008984_20170312T084235/IMG_DATA/T34JFP_20170312T082001_B08.jp2')
sp_bands = [raster_jfp_b2_fp, raster_jfp_b3_fp, raster_jfp_b4_fp, raster_jfp_b8_fp]

date = "2017-03-12"
create_mosaic(fp_bands, sp_bands, date)

first part completed
second part completed
mosaic is created and saved


In [30]:
### 2017-03-22 mosiac

raster_b2_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170322T081611_N0204_R121_T34JEP_20170322T084728.SAFE/GRANULE/L1C_T34JEP_A009127_20170322T084728/IMG_DATA/T34JEP_20170322T081611_B02.jp2')
raster_b3_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170322T081611_N0204_R121_T34JEP_20170322T084728.SAFE/GRANULE/L1C_T34JEP_A009127_20170322T084728/IMG_DATA/T34JEP_20170322T081611_B03.jp2')
raster_b4_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170322T081611_N0204_R121_T34JEP_20170322T084728.SAFE/GRANULE/L1C_T34JEP_A009127_20170322T084728/IMG_DATA/T34JEP_20170322T081611_B04.jp2')
raster_b8_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170322T081611_N0204_R121_T34JEP_20170322T084728.SAFE/GRANULE/L1C_T34JEP_A009127_20170322T084728/IMG_DATA/T34JEP_20170322T081611_B08.jp2')
fp_bands = [raster_b2_fp, raster_b3_fp, raster_b4_fp, raster_b8_fp]


raster_jfp_b2_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170322T081611_N0204_R121_T34JFP_20170322T084728.SAFE/GRANULE/L1C_T34JFP_A009127_20170322T084728/IMG_DATA/T34JFP_20170322T081611_B02.jp2')
raster_jfp_b3_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170322T081611_N0204_R121_T34JFP_20170322T084728.SAFE/GRANULE/L1C_T34JFP_A009127_20170322T084728/IMG_DATA/T34JFP_20170322T081611_B03.jp2')
raster_jfp_b4_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170322T081611_N0204_R121_T34JFP_20170322T084728.SAFE/GRANULE/L1C_T34JFP_A009127_20170322T084728/IMG_DATA/T34JFP_20170322T081611_B04.jp2')
raster_jfp_b8_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170322T081611_N0204_R121_T34JFP_20170322T084728.SAFE/GRANULE/L1C_T34JFP_A009127_20170322T084728/IMG_DATA/T34JFP_20170322T081611_B05.jp2')
sp_bands = [raster_jfp_b2_fp, raster_jfp_b3_fp, raster_jfp_b4_fp, raster_jfp_b8_fp]

date = "2017-03-22"
create_mosaic(fp_bands, sp_bands, date)

first part completed
second part completed
mosaic is created and saved


In [31]:
## 2017-05-31 mosiac

raster_b2_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170531T082011_N0205_R121_T34JEP_20170531T084246.SAFE/GRANULE/L1C_T34JEP_A010128_20170531T084246/IMG_DATA/T34JEP_20170531T082011_B02.jp2')
raster_b3_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170531T082011_N0205_R121_T34JEP_20170531T084246.SAFE/GRANULE/L1C_T34JEP_A010128_20170531T084246/IMG_DATA/T34JEP_20170531T082011_B03.jp2')
raster_b4_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170531T082011_N0205_R121_T34JEP_20170531T084246.SAFE/GRANULE/L1C_T34JEP_A010128_20170531T084246/IMG_DATA/T34JEP_20170531T082011_B04.jp2')
raster_b8_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170531T082011_N0205_R121_T34JEP_20170531T084246.SAFE/GRANULE/L1C_T34JEP_A010128_20170531T084246/IMG_DATA/T34JEP_20170531T082011_B08.jp2')
fp_bands = [raster_b2_fp, raster_b3_fp, raster_b4_fp, raster_b8_fp]


raster_jfp_b2_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170531T082011_N0205_R121_T34JFP_20170531T084246.SAFE/GRANULE/L1C_T34JFP_A010128_20170531T084246/IMG_DATA/T34JFP_20170531T082011_B02.jp2')
raster_jfp_b3_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170531T082011_N0205_R121_T34JFP_20170531T084246.SAFE/GRANULE/L1C_T34JFP_A010128_20170531T084246/IMG_DATA/T34JFP_20170531T082011_B03.jp2')
raster_jfp_b4_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170531T082011_N0205_R121_T34JFP_20170531T084246.SAFE/GRANULE/L1C_T34JFP_A010128_20170531T084246/IMG_DATA/T34JFP_20170531T082011_B04.jp2')
raster_jfp_b8_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170531T082011_N0205_R121_T34JFP_20170531T084246.SAFE/GRANULE/L1C_T34JFP_A010128_20170531T084246/IMG_DATA/T34JFP_20170531T082011_B08.jp2')
sp_bands = [raster_jfp_b2_fp, raster_jfp_b3_fp, raster_jfp_b4_fp, raster_jfp_b8_fp]

date = "2017-05-31"
create_mosaic(fp_bands, sp_bands, date)

first part completed
second part completed
mosaic is created and saved


In [32]:
## 2017-06-20 mosiac

raster_b2_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170620T082011_N0205_R121_T34JEP_20170620T084200.SAFE/GRANULE/L1C_T34JEP_A010414_20170620T084200/IMG_DATA/T34JEP_20170620T082011_B02.jp2')
raster_b3_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170620T082011_N0205_R121_T34JEP_20170620T084200.SAFE/GRANULE/L1C_T34JEP_A010414_20170620T084200/IMG_DATA/T34JEP_20170620T082011_B03.jp2')
raster_b4_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170620T082011_N0205_R121_T34JEP_20170620T084200.SAFE/GRANULE/L1C_T34JEP_A010414_20170620T084200/IMG_DATA/T34JEP_20170620T082011_B04.jp2')
raster_b8_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170620T082011_N0205_R121_T34JEP_20170620T084200.SAFE/GRANULE/L1C_T34JEP_A010414_20170620T084200/IMG_DATA/T34JEP_20170620T082011_B08.jp2')
fp_bands = [raster_b2_fp, raster_b3_fp, raster_b4_fp, raster_b8_fp]


raster_jfp_b2_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170620T082011_N0205_R121_T34JFP_20170620T084200.SAFE/GRANULE/L1C_T34JFP_A010414_20170620T084200/IMG_DATA/T34JFP_20170620T082011_B02.jp2')
raster_jfp_b3_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170620T082011_N0205_R121_T34JFP_20170620T084200.SAFE/GRANULE/L1C_T34JFP_A010414_20170620T084200/IMG_DATA/T34JFP_20170620T082011_B03.jp2')
raster_jfp_b4_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170620T082011_N0205_R121_T34JFP_20170620T084200.SAFE/GRANULE/L1C_T34JFP_A010414_20170620T084200/IMG_DATA/T34JFP_20170620T082011_B04.jp2')
raster_jfp_b8_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170620T082011_N0205_R121_T34JFP_20170620T084200.SAFE/GRANULE/L1C_T34JFP_A010414_20170620T084200/IMG_DATA/T34JFP_20170620T082011_B08.jp2')
sp_bands = [raster_jfp_b2_fp, raster_jfp_b3_fp, raster_jfp_b4_fp, raster_jfp_b8_fp]

date = "2017-06-20"
create_mosaic(fp_bands, sp_bands, date)

first part completed
second part completed
mosaic is created and saved


In [33]:
## 2017-07-10 mosiac

raster_b2_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170710T082011_N0205_R121_T34JEP_20170710T084244.SAFE/GRANULE/L1C_T34JEP_A010700_20170710T084244/IMG_DATA/T34JEP_20170710T082011_B02.jp2')
raster_b3_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170710T082011_N0205_R121_T34JEP_20170710T084244.SAFE/GRANULE/L1C_T34JEP_A010700_20170710T084244/IMG_DATA/T34JEP_20170710T082011_B03.jp2')
raster_b4_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170710T082011_N0205_R121_T34JEP_20170710T084244.SAFE/GRANULE/L1C_T34JEP_A010700_20170710T084244/IMG_DATA/T34JEP_20170710T082011_B04.jp2')
raster_b8_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170710T082011_N0205_R121_T34JEP_20170710T084244.SAFE/GRANULE/L1C_T34JEP_A010700_20170710T084244/IMG_DATA/T34JEP_20170710T082011_B08.jp2')
fp_bands = [raster_b2_fp, raster_b3_fp, raster_b4_fp, raster_b8_fp]


raster_jfp_b2_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170710T082011_N0205_R121_T34JFP_20170710T084244.SAFE/GRANULE/L1C_T34JFP_A010700_20170710T084244/IMG_DATA/T34JFP_20170710T082011_B02.jp2')
raster_jfp_b3_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170710T082011_N0205_R121_T34JFP_20170710T084244.SAFE/GRANULE/L1C_T34JFP_A010700_20170710T084244/IMG_DATA/T34JFP_20170710T082011_B03.jp2')
raster_jfp_b4_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170710T082011_N0205_R121_T34JFP_20170710T084244.SAFE/GRANULE/L1C_T34JFP_A010700_20170710T084244/IMG_DATA/T34JFP_20170710T082011_B04.jp2')
raster_jfp_b8_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170710T082011_N0205_R121_T34JFP_20170710T084244.SAFE/GRANULE/L1C_T34JFP_A010700_20170710T084244/IMG_DATA/T34JFP_20170710T082011_B08.jp2')
sp_bands = [raster_jfp_b2_fp, raster_jfp_b3_fp, raster_jfp_b4_fp, raster_jfp_b8_fp]

date = "2017-07-10"
create_mosaic(fp_bands, sp_bands, date)

first part completed
second part completed
mosaic is created and saved


In [34]:
## 2017-07-15 mosiac

raster_b2_fp = os.path.join(parent_path, 'S2B_MSIL1C_20170715T081609_N0205_R121_T34JEP_20170715T084650.SAFE/GRANULE/L1C_T34JEP_A001863_20170715T084650/IMG_DATA/T34JEP_20170715T081609_B02.jp2')
raster_b3_fp = os.path.join(parent_path, 'S2B_MSIL1C_20170715T081609_N0205_R121_T34JEP_20170715T084650.SAFE/GRANULE/L1C_T34JEP_A001863_20170715T084650/IMG_DATA/T34JEP_20170715T081609_B03.jp2')
raster_b4_fp = os.path.join(parent_path, 'S2B_MSIL1C_20170715T081609_N0205_R121_T34JEP_20170715T084650.SAFE/GRANULE/L1C_T34JEP_A001863_20170715T084650/IMG_DATA/T34JEP_20170715T081609_B04.jp2')
raster_b8_fp = os.path.join(parent_path, 'S2B_MSIL1C_20170715T081609_N0205_R121_T34JEP_20170715T084650.SAFE/GRANULE/L1C_T34JEP_A001863_20170715T084650/IMG_DATA/T34JEP_20170715T081609_B08.jp2')
fp_bands = [raster_b2_fp, raster_b3_fp, raster_b4_fp, raster_b8_fp]


raster_jfp_b2_fp = os.path.join(parent_path, 'S2B_MSIL1C_20170715T081609_N0205_R121_T34JFP_20170715T084650.SAFE/GRANULE/L1C_T34JFP_A001863_20170715T084650/IMG_DATA/T34JFP_20170715T081609_B02.jp2')
raster_jfp_b3_fp = os.path.join(parent_path, 'S2B_MSIL1C_20170715T081609_N0205_R121_T34JFP_20170715T084650.SAFE/GRANULE/L1C_T34JFP_A001863_20170715T084650/IMG_DATA/T34JFP_20170715T081609_B03.jp2')
raster_jfp_b4_fp = os.path.join(parent_path, 'S2B_MSIL1C_20170715T081609_N0205_R121_T34JFP_20170715T084650.SAFE/GRANULE/L1C_T34JFP_A001863_20170715T084650/IMG_DATA/T34JFP_20170715T081609_B04.jp2')
raster_jfp_b8_fp = os.path.join(parent_path, 'S2B_MSIL1C_20170715T081609_N0205_R121_T34JFP_20170715T084650.SAFE/GRANULE/L1C_T34JFP_A001863_20170715T084650/IMG_DATA/T34JFP_20170715T081609_B08.jp2')
sp_bands = [raster_jfp_b2_fp, raster_jfp_b3_fp, raster_jfp_b4_fp, raster_jfp_b8_fp]

date = "2017-07-15"
create_mosaic(fp_bands, sp_bands, date)

first part completed
second part completed
mosaic is created and saved


In [35]:
## 2017-08-04 mosiac

raster_b2_fp = os.path.join(parent_path, 'S2B_MSIL1C_20170804T081559_N0205_R121_T34JEP_20170804T084631.SAFE/GRANULE/L1C_T34JEP_A002149_20170804T084631/IMG_DATA/T34JEP_20170804T081559_B02.jp2')
raster_b3_fp = os.path.join(parent_path, 'S2B_MSIL1C_20170804T081559_N0205_R121_T34JEP_20170804T084631.SAFE/GRANULE/L1C_T34JEP_A002149_20170804T084631/IMG_DATA/T34JEP_20170804T081559_B03.jp2')
raster_b4_fp = os.path.join(parent_path, 'S2B_MSIL1C_20170804T081559_N0205_R121_T34JEP_20170804T084631.SAFE/GRANULE/L1C_T34JEP_A002149_20170804T084631/IMG_DATA/T34JEP_20170804T081559_B04.jp2')
raster_b8_fp = os.path.join(parent_path, 'S2B_MSIL1C_20170804T081559_N0205_R121_T34JEP_20170804T084631.SAFE/GRANULE/L1C_T34JEP_A002149_20170804T084631/IMG_DATA/T34JEP_20170804T081559_B08.jp2')
fp_bands = [raster_b2_fp, raster_b3_fp, raster_b4_fp, raster_b8_fp]


raster_jfp_b2_fp = os.path.join(parent_path, 'S2B_MSIL1C_20170804T081559_N0205_R121_T34JFP_20170804T084631.SAFE/GRANULE/L1C_T34JFP_A002149_20170804T084631/IMG_DATA/T34JFP_20170804T081559_B02.jp2')
raster_jfp_b3_fp = os.path.join(parent_path, 'S2B_MSIL1C_20170804T081559_N0205_R121_T34JFP_20170804T084631.SAFE/GRANULE/L1C_T34JFP_A002149_20170804T084631/IMG_DATA/T34JFP_20170804T081559_B03.jp2')
raster_jfp_b4_fp = os.path.join(parent_path, 'S2B_MSIL1C_20170804T081559_N0205_R121_T34JFP_20170804T084631.SAFE/GRANULE/L1C_T34JFP_A002149_20170804T084631/IMG_DATA/T34JFP_20170804T081559_B04.jp2')
raster_jfp_b8_fp = os.path.join(parent_path, 'S2B_MSIL1C_20170804T081559_N0205_R121_T34JFP_20170804T084631.SAFE/GRANULE/L1C_T34JFP_A002149_20170804T084631/IMG_DATA/T34JFP_20170804T081559_B08.jp2')
sp_bands = [raster_jfp_b2_fp, raster_jfp_b3_fp, raster_jfp_b4_fp, raster_jfp_b8_fp]

date = "2017-08-04"
create_mosaic(fp_bands, sp_bands, date)

first part completed
second part completed
mosaic is created and saved


In [ ]:
## 2017-08-19 mosiac

raster_b2_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170819T082011_N0205_R121_T34JEP_20170819T084427.SAFE/GRANULE/L1C_T34JEP_A011272_20170819T084427/IMG_DATA/T34JEP_20170819T082011_B02.jp2')
raster_b3_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170819T082011_N0205_R121_T34JEP_20170819T084427.SAFE/GRANULE/L1C_T34JEP_A011272_20170819T084427/IMG_DATA/T34JEP_20170819T082011_B03.jp2')
raster_b4_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170819T082011_N0205_R121_T34JEP_20170819T084427.SAFE/GRANULE/L1C_T34JEP_A011272_20170819T084427/IMG_DATA/T34JEP_20170819T082011_B04.jp2')
raster_b8_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170819T082011_N0205_R121_T34JEP_20170819T084427.SAFE/GRANULE/L1C_T34JEP_A011272_20170819T084427/IMG_DATA/T34JEP_20170819T082011_B08.jp2')
fp_bands = [raster_b2_fp, raster_b3_fp, raster_b4_fp, raster_b8_fp]


raster_jfp_b2_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170819T082011_N0205_R121_T34JFP_20170819T084427.SAFE/GRANULE/L1C_T34JFP_A011272_20170819T084427/IMG_DATA/T34JFP_20170819T082011_B02.jp2')
raster_jfp_b3_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170819T082011_N0205_R121_T34JFP_20170819T084427.SAFE/GRANULE/L1C_T34JFP_A011272_20170819T084427/IMG_DATA/T34JFP_20170819T082011_B03.jp2')
raster_jfp_b4_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170819T082011_N0205_R121_T34JFP_20170819T084427.SAFE/GRANULE/L1C_T34JFP_A011272_20170819T084427/IMG_DATA/T34JFP_20170819T082011_B04.jp2')
raster_jfp_b8_fp = os.path.join(parent_path, 'S2A_MSIL1C_20170819T082011_N0205_R121_T34JFP_20170819T084427.SAFE/GRANULE/L1C_T34JFP_A011272_20170819T084427/IMG_DATA/T34JFP_20170819T082011_B08.jp2')
sp_bands = [raster_jfp_b2_fp, raster_jfp_b3_fp, raster_jfp_b4_fp, raster_jfp_b8_fp]

date = "2017-08-19"
create_mosaic(fp_bands, sp_bands, date)

first part completed


In [ ]:
train_fe['area'] = train_fe['geometry'].area
train_fe['centroid'] = train_fe['geometry'].centroid
test_fe['area'] = test_fe['geometry'].area
test_fe['centroid'] = test_fe['geometry'].centroid